In [1]:
%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
#from scipy.signal import butter, lfilter
#import scipy.ndimage
import os
import argparse

#import spectrograms.spectrogram_utils as su

#import winsound
import pandas as pd

import glob as gl
from IPython.display import display, HTML

import pydub as pyd
import spectrogram as sp
from utils import generate_merlin_wav

from data import *

import shutil as sh

import evaluate_loss_func_for_notebook as el

%load_ext autoreload
%autoreload 2

vctk_raw_folder = '/home/ubuntu/VCTK-Corpus/'
vctk_prebuilt_folder = '/home/ubuntu/vctk-16khz-cmu-no-boundaries/'

/usr/local/lib/python2.7/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [5]:
def get_vtck_speaker_info():
    # this is the reference data file that comes as part of VCTK
    speaker_info_file = os.path.join(vctk_raw_folder, 'speaker-info.txt')

   # read file contents
    f = open(speaker_info_file, 'r')
    x = f.readlines()
    f.close()

    # extract column headers
    cols = x[0].lower().split()
    num_cols = len(cols)
    del x[0]
    num_speaker = len(x)

    # parse the data line by line
    d = dict()

    for idx in range(num_speaker):
        this_speaker_id = int(x[idx][:3])
        this_age = int(x[idx][5:7])
        this_gender = x[idx][9]
        residual = x[idx][14:].split()
        this_accent = residual[0]
        this_region = " ".join(residual[1:])

        # add speakers to a dictionary
        d[this_speaker_id] = (this_speaker_id, this_age, this_gender, this_accent, this_region)

    # convert to Pandas datafrae
    speaker_info = pd.DataFrame.from_dict(d, orient='index', columns=cols)
    
    return speaker_info

In [6]:
# load VCTK speaker_info metadata
speaker_info = get_vtck_speaker_info()

In [7]:
### Creating a Dataset for all VCTK
all_vctk_speakers = speaker_info[speaker_info.id != 280]

In [8]:
feat_files = [os.path.basename(g)[:-4] for g in gl.glob(os.path.join('/home/ubuntu/vctk-16khz-cmu-no-boundaries/numpy_features', '*.npz'))]

In [9]:
len(feat_files)

44039

In [10]:
all_spkr_id = [int(f.split('_')[0][1:]) for f in feat_files]

In [11]:
all_vctk_spkr_id = all_vctk_speakers['id'].values

In [12]:
num_all_vctk = len(all_vctk_spkr_id)

In [13]:
num_samples_for_speaker = np.zeros(num_all_vctk)
num_samples_for_speaker_valid = np.zeros(num_all_vctk, dtype=np.int8)
train_samples_for_speaker = []
valid_samples_for_speaker = []

for i, spkr in enumerate(all_vctk_spkr_id):
    idx_this_spkr = np.array([j == spkr for j in all_spkr_id])
    num_samples_for_speaker[i] = np.sum(idx_this_spkr)
    
    num_samples_for_speaker_valid[i] = np.int(np.floor(num_samples_for_speaker[i] * 0.1))
    
    flat_idx = np.flatnonzero(idx_this_spkr)
    np.random.shuffle(flat_idx)
    
    valid_samples_for_speaker.append(flat_idx[:num_samples_for_speaker_valid[i]])
    train_samples_for_speaker.append(flat_idx[num_samples_for_speaker_valid[i]:])    
    

In [14]:
np.sum([len(x) for x in valid_samples_for_speaker])

4316

In [15]:
np.sum([len(x) for x in train_samples_for_speaker])

39313

In [20]:
src_folder = '/home/ubuntu/vctk-16khz-cmu-no-boundaries/numpy_features'
dest_folder = '/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all'
feat_files_array = np.array(feat_files)

for i, spkr in enumerate(all_vctk_spkr_id):
    f = 'numpy_features'
    #f = 'numpy_features_valid'
    this_dest = os.path.join(dest_folder, f)
    
    try:
        os.makedirs(this_dest)
    except:
        pass
    
    for t in feat_files_array[train_samples_for_speaker[i]]:
    #for t in feat_files_array[valid_samples_for_speaker[i]]:
        this_src = os.path.join(src_folder, t+'.npz')
        this_dest_file = os.path.join(this_dest, t+'.npz')
        #print this_src, this_dest
        # also need to convert the floats... then copy
        
        try:
            tmp = dict(np.load(this_src))
            tmp['audio_features'] = tmp['audio_features'].astype(np.float32)
            tmp['audio_norminfo'] = tmp['audio_norminfo'].astype(np.float32)
            
            save_dict = {
                    'audio_features': tmp['audio_features'],
                    'code2phone': tmp['code2phone'],
                    'phonemes': tmp['phonemes'],
                    'language': tmp['language'],
                    'audio_norminfo': tmp['audio_norminfo'],
                }
            
            np.savez_compressed(this_dest_file, **tmp)
            #np.savez(dest_file, tmp)
        except:
            print "Couldn't copy: " + src_file

In [23]:
print len(os.listdir(os.path.join(dest_folder, 'numpy_features')))
print len(os.listdir(os.path.join(dest_folder, 'numpy_features_valid')))

39313
4316


In [ ]:
print 6856 + 751

In [ ]:
len(feat_files)

In [27]:
np.sum([np.sum(all_vctk_spkr_id == s) for s in all_spkr_id])

43629

In [35]:
gl.glob(os.path.join('/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-us/numpy_features', 'p315*.npz'))

[]

#### Copy norm_info file

In [17]:
sh.copytree('/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-us/norm_info', '/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info')